In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from dateutil.relativedelta import relativedelta
from tqdm import tqdm #tqdm: 반복문 돌릴때 진행상태 보여주는 패키지

# Step 1. ELS자체 정보 가져오기

## 중도 상환 날짜 구현

In [2]:
#중도상환 날짜를 정수형으로 받아 옴
#datetime.date.toordinal: 각 날짜를 정수로 표현: 서기 1년 1월 1일로부터 몇번째 날인지
n0 = dt.date(2023, 4, 20) # 최초기준가격일
n1 = dt.date(2023,10, 20) # 1차
n2 = dt.date(2024, 4, 19) # 2차
n3 = dt.date(2024,10, 18) # 3차
n4 = dt.date(2025, 4, 17) # 4차
n5 = dt.date(2025,10, 20) # 5차
n6 = dt.date(2026, 4, 20) # 만기

우리는 n0부터 n6까지 매일매일 가상의 주가를 만들어서 ELS수익률을 계산할 것.

In [3]:
#날짜 간의 차 (relativedelta)를 만들고, 여기서 날짜에 해당하는 숫자만 뽑아 줌. 중도상환 날짜를 숫자로 변환해 두는 것
#이 숫자는 나중에 중도상환일의 주가를 찾을 때 사용될 것임
check_day = np.array([(n1 - n0).days, (n2 - n0).days, (n3 - n0).days, (n4 - n0).days, (n5 - n0).days, (n6 - n0).days])
check_day

array([ 183,  365,  547,  728,  914, 1096])

In [4]:
tot_date = (n6 - n0).days #만기까지의 전체 기간: 이 값은 주가 시뮬레이션 할 전체 길이를 계산하는 데 사용될 것임
dt = 1 / 365

## 기초자산 정보 구현 (가격변화 제외)

In [5]:
face_value = 10 ** 4 #액면가는 1만원

In [6]:
r=0.0165 #무위험수익률
x_vol=0.2662 #자산 x의 시그마
y_vol=0.2105 #자산 y의 시그마
z_vol=0.2111 #자산 z의 시그마
rho_xy, rho_xz, rho_yz = 0.279, 0.2895, 0.5256 #각 기초자산 간의 상관계수

In [7]:
#상관계수의 정보를 모두 담은 행렬을 만들어주고
corr = np.array([[1, rho_xy, rho_xz],
                 [rho_xy, 1, rho_yz],
                 [rho_xz, rho_yz, 1]])

#이걸 콜레스키 분해로 Lower Triangular Matrix로 변환해 줌
k = np.linalg.cholesky(corr)
k

array([[1.        , 0.        , 0.        ],
       [0.279     , 0.9602911 , 0.        ],
       [0.2895    , 0.4632236 , 0.83762381]])

In [8]:
strike_price = np.array([0.95, 0.90, 0.85, 0.85, 0.85, 0.75]) #각 상환시점 별 조기상환의 기준이 되는 수익률
mature_return=0.0896 #조기상환 안 될 때 약정수익률: 연 8.96%
coupon_rate = np.array([mature_return*0.5, mature_return*1, mature_return*1.5, mature_return*2, mature_return*2.5, mature_return*3])
#coupon_rate: 각 시점마다 조기상환되었을 경우 약정수익률을 계산해 둔 리스트
kib = 0.5 #낙인배리어 50%

# Step 2. 수익률 시뮬레이션 이해하기
Step 2에서는 이 정보를 바탕으로 ELS 상환 시뮬레이션 구조를 따라가 보도록!

In [9]:
#종목 1, 2, 3의 3년간 매일 주가 정보를 담을 빈 리스트
S1= np.zeros((tot_date + 1, 1))
S2=np.zeros((tot_date + 1, 1))
S3=np.zeros((tot_date + 1, 1)) 
S1.shape

(1097, 1)

In [10]:
#결국 낙인베리어 및 조기상환 여부는 약관된 수익률을 갖고 결정되니까, 처음 가격을 100%로 가정하겠음
S1[0], S2[0], S3[0] = 1, 1, 1

In [11]:
ratio_S1, ratio_S2, ratio_S3 = S1[0], S2[0], S3[0]

큰 플로우는 이렇게 구성됨:
1. 쿠폰율대로 상환될 경우 고객에게 지급될 금액을 계산해 둠
2. 시뮬레이션된 가상주가 생성
3. 각 상환 시점에서 예상되는 상환액을 계산함
    * 조기상환 조건 충족 시 1번에서 계산한 쿠폰율대로 상환함
    * 조기상환 안 되었지만 다행히 낙인배리어 안 찍은 경우 약정수익률대로 상환함
    * 둘 다 아니면 낙인배리어 찍었으므로, 기준일 대비 상환일에 주가가 가장 많이 떨어진 종목의 수익률대로 상환됨

In [12]:
repay_n = len(strike_price) #상환 여부가 결정되는 시점은 앞에서 정한 6개 시점임

## 2-1

In [13]:
early_payment= np.zeros([repay_n, 1]) #각 시점에서 쿠폰율대로 조기상환될 경우 고객에게 지급될 금액
for j in range(repay_n):
    early_payment[j] = face_value * (1 + coupon_rate[j]) #시점별 고객에게 주는 상환액(payment) 계산: 1만원*쿠폰율
early_payment

array([[10448.],
       [10896.],
       [11344.],
       [11792.],
       [12240.],
       [12688.]])

2~4번은 1만번 시뮬레이션 해서 그 평균값을 도출하는 방법을 사용할 것임

일단 한번의 시뮬레이션 과정을 팔로우하고 그 뒤에 1만번 반복해보자

수업 중에 1만번은 무리고 천번 정도만 하는게 맞을거 같긴 함


In [14]:
np.random.seed(48) #랜덤시드 박고

## 2-2
주가 수익률이 정규분포를 따른다는 가정 하에 BSM으로 주가 시뮬

In [15]:
#자산 3개에 대해 정규분포를 따르는 수열 생성: 일평균 수익률이 될 것임
w0 = np.random.normal(0, 1, size=[tot_date, 3]) 
w0 = np.transpose(w0) #(3,t)
w0

array([[-0.99363132,  1.06158869, -2.07485402, ...,  0.15383183,
        -0.50740802,  2.16631788],
       [-1.06340081, -0.15740415,  0.70063301, ...,  0.01857837,
         1.97777285,  0.88310094],
       [-0.63807592,  1.09372848,  1.67463863, ..., -0.17348662,
        -1.25897671, -1.46037467]])

In [16]:
#여기에 자산 간 상관계수 정보 담겨있는 cholesky까지 곱해줌
#즉 자산 개별 및 자산 간 변동성까지 고려한, 데일리 주가 수익률 = w가 된다.
w = np.matmul(k, w0)
w

array([[-0.99363132,  1.06158869, -2.07485402, ...,  0.15383183,
        -0.50740802,  2.16631788],
       [-1.29839747,  0.14502944,  0.09392737, ...,  0.06075973,
         1.75767083,  1.45243667],
       [-1.31471619,  1.15054962,  1.1265967 , ..., -0.09217626,
        -0.28529244, -0.18702237]])

이제 BSM에 w를 이용하여, 각 3개 주식의, 3년간의 주가 정보 리스트를 만듦

In [17]:
for j in range(tot_date): # 평가기간 동안 똑같이 GBM으로 난수생성: 주가 시뮬
    S1[j + 1] = S1[j] * np.exp((r - 0.5 * x_vol ** 2) * dt + x_vol * np.sqrt(dt) * w[0, j])
    S2[j + 1] = S2[j] * np.exp((r - 0.5 * y_vol ** 2) * dt + y_vol * np.sqrt(dt) * w[1, j])
    S3[j + 1] = S3[j] * np.exp((r - 0.5 * z_vol ** 2) * dt + z_vol * np.sqrt(dt) * w[2, j])

S1

array([[1.        ],
       [0.98619944],
       [1.00084351],
       ...,
       [0.97275053],
       [0.96584735],
       [0.99539378]])

## 2-3
예상 상환액 계산

worst performer를 기준으로 상환 기준 위에 있으면 조기상환되므로 worst performer를 확인해야 함

In [18]:
WP = np.minimum(S1, S2, S3)
WP

array([[1.        ],
       [0.98578073],
       [0.98734192],
       ...,
       [0.9133222 ],
       [0.93116769],
       [0.94617443]])

조기상환되는 날의 worst performer만 뽑자

In [19]:
WP_checkday = WP[check_day]
WP_checkday 

array([[0.88024477],
       [1.15480757],
       [1.05280836],
       [0.88847554],
       [0.82546806],
       [0.94617443]])

payoff: 시뮬 1회 후, 상환 가능 시점 6개에서 예상되는 상환금액을 담아 둘 벡터

지금은 아무 값도 없으므로 그냥 0으로 지정

In [20]:
payoff = np.zeros([repay_n, 1])
payoff

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

조기상환 여부를 알려주는 개체를 생성해서,

만약 조기상환이 되면 그 개체를 1이라 하고 아니라면 0으로 해 둘것임

In [22]:
repay_event = 0 #일단 조기상환이 되지 않았다고 가정

for j in range(repay_n): #6번의 기준일마다 다 상환되는지 확인
    if WP_checkday[j] >= strike_price[j]: #상환일의 가장 떨어진 기초자산이 행사가격보다 위에 있으면, 즉 조기상환 조건 달성하면
        repay_event = 1 #상환완료 처리
        payoff[j] = early_payment[j] # 위에서 구한 조기상환 되었을 때의 가격을 payoff 에 넣고
        break #상환됐으니까 for문 깨고 나옴. 다음 payoff들은 알 필요 없음. 아래 if문은 실행되지 않음

if repay_event == 0: #6개 상환 시점 다 확인했는데도 상환이 발생하지 않았다면 이 부분에서 시뮬됨
    if np.min(WP) > kib: #worst performance 중에서 가장 낮은 것도 kib보다 위, 즉 낙인배리어 찍은적 없는데 상환은 안된경우 - 약정된 수익률대로 상환
        dummy=strike_price[-1] #이때 약정된 수익률은 3년 만기 수익률이죠?
        payoff[-1] = face_value * (1 + dummy) #미리 정해져있던 수익률로 상환
    else: #낙인배리어 아래에 한번이라도 찍었고, 조기상환도 안 된 최악의 경우
        payoff[-1] = face_value * WP[-1] #최종가격기준결정일에 가장 떨어진 기초자산이 떨어진만큼 손실 발생

payoff #이렇게 한 번 시뮬 마쳤을 때 상환 결과: 이번에는 1년차에서 조기상환되었음

array([[    0.],
       [10896.],
       [    0.],
       [    0.],
       [    0.],
       [    0.]])

# Step 3. 반복 시뮬을 통한 적정가 도출
Step 2에서 한 시뮬레이션을 1만번 반복하여 ELS의 적정가치를 계산

플로우는 아래와 같다:

    2-1 (조기상환시 지급액)은 계산해 뒀으니 패스하고
    아래 내용을 1000번 반복
    * 2-2 (BSM 시뮬)
    * 2-3 (시뮬된 주가에 따른 예상 상환액 계산)
    그렇게 해서 나온 6번 지급시점동안의 상환액을 계속 누적해서 더해 줌

In [42]:
n=1000 #주가 시뮬레이션 횟수. 사실 10만번 이상 해야하는데 시간관계상 1000번만

In [43]:
#S1 S2 S3 초기화
S1=np.zeros((tot_date + 1, 1))
S2=np.zeros((tot_date + 1, 1))
S3=np.zeros((tot_date + 1, 1))
S1[0], S2[0], S3[0] = 1, 1, 1

In [44]:
tot_payoff = np.zeros([repay_n, 1]) #모든 시뮬 결과는 여기다가 다 더해집니다.
tot_payoff

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [47]:
for _ in tqdm(range(n)): #아래 과정 전체를 n번 반복. tqdm으로 감싸주면 진행률 나온다.
    #2-2번 내용
    w0 = np.random.normal(0, 1, size=[tot_date, 3])
    w0 = np.transpose(w0)
    w = np.matmul(k, w0)
    for j in range(tot_date): # 똑같이 GBM으로 난수생성
        S1[j + 1] = S1[j] * np.exp((r - 0.5 * x_vol ** 2) * dt + x_vol * np.sqrt(dt) * w[0, j])
        S2[j + 1] = S2[j] * np.exp((r - 0.5 * y_vol ** 2) * dt + y_vol * np.sqrt(dt) * w[1, j])
        S3[j + 1] = S3[j] * np.exp((r - 0.5 * z_vol ** 2) * dt + z_vol * np.sqrt(dt) * w[2, j])
    #2-3번 내용
    WP = np.minimum(S1, S2, S3)
    WP_checkday = WP[check_day]
    payoff = np.zeros([repay_n, 1])
    repay_event = 0
    for j in range(repay_n):
        if WP_checkday[j] >= strike_price[j]:
            payoff[j] = early_payment[j] 
            repay_event = 1
            break 
    if repay_event == 0: 
        if np.min(WP) > kib:
            dummy=strike_price[-1]
            payoff[-1] = face_value * (1 + dummy) 
        else: 
            payoff[-1] = face_value * WP[-1]

    tot_payoff += payoff #tot_payoff에 매 payoff를 더해 줌. 6개 값이 각각 더해짐 이걸 나중에 평균해서 평균 지급액을 구할거에요

100%|██████████| 1000/1000 [00:18<00:00, 54.70it/s]


In [48]:
tot_payoff

array([[4210544.        ],
       [1906800.        ],
       [ 986928.        ],
       [ 400928.        ],
       [ 306000.        ],
       [2785146.40867397]])

In [51]:
#시뮬레이션 돌린 전체 지급액을 반복횟수 n으로 나눕니다.
#즉 n번 몬테카를로 시행후, 시점별로 받을 수 있는 상환액의 평균 나옴
mean_payoff = tot_payoff / n
mean_payoff

array([[4210.544     ],
       [1906.8       ],
       [ 986.928     ],
       [ 400.928     ],
       [ 306.        ],
       [2785.14640867]])

In [52]:
discount_payoff=np.zeros([repay_n, 1]) #빈 벡터 또 하나 만듦: 현가로 할인된 평균상환액
for j in range(repay_n): # 각 시점의 평균상환액을 무위험이자율로 할인해서 현재가치 도출
    discount_payoff[j] = mean_payoff[j] * np.exp(-r * check_day[j] / 365)
discount_payoff

array([[4175.855522  ],
       [1875.59594142],
       [ 962.8230933 ],
       [ 387.94837282],
       [ 293.61438587],
       [2650.51838264]])

In [53]:
price = np.sum(discount_payoff) #ELS의 현가 = 들어올 수 있는 현재가치의 합 NPV
print(price)

10346.35569804402


즉 우리는 10346원의 기대수익률을 갖는 상품을 1만원에 사는 것!